![alt text](https://zewailcity.edu.eg/main/images/logo3.png)

_Prepared by_  [**Muhammad Hamdy AlAref**](mailto:malaref@zewailcity.edu.eg)
$\DeclareMathOperator*{\argmax}{argmax} % thin space, limits underneath in displays$
<!-- needed for \argmax command in LaTeX below -->

# Decision Processes (DPs)

A decision process is a type of problem that requires _sequential decisions_. Generally, a decision process need not be determinstic, and that's why a _solution_ to one usually takes the form of a _policy_. A _policy_ $\pi(s)$ can be viewed as a function that maps any state to an action. That is different from classical search algorithms that consider a subset of the state space to find a _path_ to a _goal_.

Nonetheless, to better understand decision processes, we are going to start with a deterministic decision process. This will lay the needed to groundwork for the famous Markovian Decision Processes (MDPs) later on.

## Visualization

As always, we have some code to help us visualize the problem! So let's get that out of the way!

In [ ]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0

    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for state in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, state)
        print('-' * terminal_width)

def _tom_and_jerry_visualizer(problem, state):
    '''Custom visualizer for the sudoku puzzle CSP.'''
    for j in range(problem.bounds[1] - 1, -1, -1):
        for i in range(problem.bounds[0]):
            print(('%8s' if type(state[(i, j)]) is str else '%8.2f') % state[(i, j)], end='\t')
        print()

## Deterministic Formulation

We formulate a deterministic decision process using our usual problem formulation with a couple of key differences. First, we define a state space instead of an initial state, as the objective is to find a _policy_ for each one. Second, we define a **reward** function for each state. This function is very similar to the _heuristic_ function we discussed before. It returns a reward for reaching a certain state.

In [ ]:
class DDP:
    '''
    Abstract base class for a deterministic decision process formulation.
    It declares the expected methods to be used to solve it.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''

    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the state space.'''
        self.states = set()

    def actions(self, state):
        '''Returns an iterable with the applicable actions to the given state.'''
        raise NotImplementedError

    def result(self, state, action):
        '''Returns the resulting state from applying the given action to the given state.'''
        raise NotImplementedError

    def reward(self, state):
        '''Returns the reward of reaching in a certain state. Can be thought of as the heuristic of the state.'''
        raise NotImplementedError

Now that we have a formulation, we have to introduce a couple of _very important_ concepts related to decision processes!
- A **decision** is choosing an action from the available action in a state.
- An **episode** is a series of states connected by decisions (actions) starting from an initial state till a terminal state.
- The **utility** $U(s)$ of a state $s$ is defined as the expected total sum of rewards collected, starting from the reward of reaching the state to all future states till the end of a episode.
- A **discount** $\gamma$ is a penalty applied to future rewards to prefer getting them sooner than later. It has an important role in the convergence of some algorithms.
- A **Q-value** $Q(s,a)$ of a state $s$ and an action $a \in Actions(s)$ is defined as the expected total sum of rewards collected, starting from the reward of reaching the state _and choosing $a$ as the next action_ to all future states till the end of a episode.

More annotations we will be using!
- $r_s \equiv Reward(s)$
- $s' \equiv Result(s,a)$
- $A_s = Actions(s)$

### Deterministic Value Iteration

A straightforward way to find the optimal policy is to _find the utilities for all the states_. Then, an optimal policy at any state would be to choose the action that results in the highest next one. This can be done by defining the utilities using the following **Bellman equation** and solving it iteratively.

$$U_{i+1}(s) = r_s + \gamma \times \max_{a \in A_s} U_{i}(s')$$

$$\pi^*(s) = \argmax_{a \in A_s} U_{i}(s')$$

In [ ]:
def deterministic_value_iteration(ddp, init_utility=None, error=1e-6, verbose=False):
    if verbose: visualizer = Visualizer(ddp)
    utility = init_utility if init_utility is not None else {state: 0 for state in ddp.states | {None}}
    converged = False
    while not converged:
        if verbose: visualizer.visualize([utility])
        old_utility, converged = utility.copy(), True
        for state in ddp.states:
            utility[state] = ddp.reward(state) + ddp.discount * max(old_utility[ddp.result(state, action)] for action in ddp.actions(state))
            if abs(utility[state] - old_utility[state]) > error: converged = False
    return utility

def deterministic_policy_from_values(ddp, utility, verbose=True):
    if verbose: visualizer = Visualizer(ddp)
    policy = {}
    for state in ddp.states:
        policy[state] = max((action for action in ddp.actions(state)), key=lambda action: utility[ddp.result(state, action)])
    if verbose: visualizer.visualize([policy])
    return policy

### Deterministic Policy Iteration

Usually, value iteration takes more time to converge than needed to find the optimal policy. As our target is to find the optimal policy, we may start looking for it directly. That is, instead of iterating till the values converge, we **choose a policy** (can be random), use (a simpler version of) value iteration to **evaluate it**, **improve it** a bit, and **iterate** till no more improvements are possible. In effect, we will iterate through policies using the utility calculations as a guide!

$$U_{i+1}(s) = r_s + \gamma \times U_{i}(s')$$

$$\pi_{i+1}(s) = \argmax_{a \in A_s} U_{i+1}(s')$$

In [ ]:
from random import choice

def deterministic_policy_evaluation(ddp, policy, init_utility=None, error=1e-6, verbose=False):
    if verbose: visualizer = Visualizer(ddp)
    utility = init_utility if init_utility is not None else {state: 0 for state in ddp.states | {None}}
    converged = False
    while not converged:
        if verbose: visualizer.visualize([utility])
        old_utility, converged = utility.copy(), True
        for state in ddp.states:
            utility[state] = ddp.reward(state) + ddp.discount * old_utility[ddp.result(state, policy[state])]
            if abs(utility[state] - old_utility[state]) > error: converged = False
    return utility

def deterministic_policy_iteration(ddp, verbose=False):
    if verbose: visualizer = Visualizer(ddp)
    policy = {state: choice(ddp.actions(state)) for state in ddp.states}
    converged, utility = False, None
    while not converged:
        if verbose: visualizer.visualize([policy])
        converged = True
        utility = deterministic_policy_evaluation(ddp, policy, utility, verbose=verbose)
        for state in ddp.states:
            for action in ddp.actions(state):
                if utility[ddp.result(state, action)] > utility[ddp.result(state, policy[state])]:
                    policy[state] = action
                    converged = False
    return policy

### Deterministic Q-Value Iteration

A different approach to solving a decision process is to calculate the Q-values instead of the utilities (value iteration). Then, the optimal policy would be choose the action the maximizes $Q(s,a)$ at any state $s$.

$$Q_{i+1}(s, a) = r_s + \gamma \times U_{i}(s')$$
$$\equiv$$
$$Q_{i+1}(s, a) = r_s + \gamma \times \max_{a' \in A_{s'}} Q_{i}(s', a')$$
$$\pi^*(s) = \argmax_{a \in A_s} Q(s,a)$$

In [ ]:
def deterministic_q_value_iteration(ddp, init_q=None, error=1e-6):
    q = init_q if init_q is not None else {(state, action): 0 for state in ddp.states for action in ddp.actions(state)}
    converged = False
    while not converged:
        old_q, converged = q.copy(), True
        for state in ddp.states:
            for action in ddp.actions(state):
                next_state = ddp.result(state, action)
                next_state_utility = max((old_q[(next_state, next_action)] for next_action in ddp.actions(next_state)), default=0)
                q[(state, action)] = ddp.reward(state) + ddp.discount * next_state_utility
                if abs(q[(state, action)] - old_q[(state, action)]) > error: converged = False
    return q

def deterministic_policy_from_q(ddp, q, verbose=True):
    if verbose: visualizer = Visualizer(ddp)
    policy = {}
    for state in ddp.states:
        policy[state] = max((action for action in ddp.actions(state)), key=lambda action: q[(state, action)])
    if verbose: visualizer.visualize([policy])
    return policy

### Example: Tom & Jerry

Out running example for this notebook (and the next) will be our favorite cartoon [Tom and Jerry](https://en.wikipedia.org/wiki/Tom_and_Jerry)! As always, Tom is trying to catch Jerry and getting tired of Jerry getting away! Our job is to help him! (Also, Spike is sleeping and we don't want to wake him up!)

In our first (deterministic) formulation of their world, Jerry is not moving (may be sleeping or something). Note that this can be solved by simple search algorithm.

In [ ]:
class TomAndJerry(DDP):
    '''A deterministic Tom & Jerry world'''

    def __init__(self, tom, jerry, spike, bounds, max_reward, discount):
        self.tom = tom
        self.jerry = jerry
        self.spike = spike
        self.bounds = bounds
        self.max_reward = max_reward
        self.discount = discount
        self.states = {(i, j) for i in range(bounds[0]) for j in range(bounds[1])}

    def actions(self, state):
        if state is None: return []
        if state == self.jerry: return ['catch!']
        if state == self.spike: return ['ouch!']
        return ['up', 'down', 'left', 'right']

    def result(self, state, action):
        if action ==    'up': return (state[0], min(state[1] + 1, self.bounds[1] - 1))
        if action ==  'down': return (state[0], max(state[1] - 1, 0))
        if action ==  'left': return (max(state[0] - 1, 0), state[1])
        if action == 'right': return (min(state[0] + 1, self.bounds[0] - 1), state[1])
        return None

    def reward(self, state):
        if state == self.jerry: return  self.max_reward
        if state == self.spike: return -self.max_reward
        return 0

_visualizers[TomAndJerry] = _tom_and_jerry_visualizer

In [ ]:
tom_and_jerry = TomAndJerry((0, 0), (5, 5), (5, 4), (7, 7), 1000, 0.1)

In [ ]:
values = deterministic_value_iteration(tom_and_jerry, verbose=True)
deterministic_policy_from_values(tom_and_jerry, values, verbose=True)

Frontier at step 1

    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
----------------------------------------------------------------------------------------------------
Frontier at step 2

    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	 1000.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	-1000.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	

{(4, 0): 'up',
 (3, 4): 'up',
 (4, 3): 'up',
 (3, 1): 'up',
 (5, 4): 'ouch!',
 (4, 6): 'down',
 (5, 1): 'up',
 (0, 2): 'up',
 (0, 5): 'right',
 (2, 2): 'up',
 (1, 0): 'up',
 (1, 6): 'down',
 (2, 5): 'right',
 (1, 3): 'up',
 (6, 2): 'up',
 (6, 5): 'left',
 (4, 2): 'up',
 (3, 0): 'up',
 (4, 5): 'right',
 (3, 3): 'up',
 (5, 0): 'up',
 (5, 6): 'down',
 (3, 6): 'down',
 (5, 3): 'left',
 (0, 1): 'up',
 (2, 4): 'up',
 (1, 2): 'up',
 (0, 4): 'up',
 (2, 1): 'up',
 (1, 5): 'right',
 (6, 1): 'up',
 (6, 4): 'up',
 (3, 2): 'up',
 (4, 1): 'up',
 (3, 5): 'right',
 (5, 2): 'up',
 (4, 4): 'up',
 (5, 5): 'catch!',
 (0, 0): 'up',
 (1, 1): 'up',
 (0, 3): 'up',
 (2, 0): 'up',
 (1, 4): 'up',
 (0, 6): 'down',
 (2, 3): 'up',
 (2, 6): 'down',
 (6, 0): 'up',
 (6, 6): 'down',
 (6, 3): 'up'}

In [ ]:
deterministic_policy_iteration(tom_and_jerry, verbose=True)

Frontier at step 1

      up	   right	   right	   right	   right	    left	      up	
      up	    left	   right	    left	    down	  catch!	   right	
   right	    left	   right	    left	   right	   ouch!	      up	
    left	    left	    left	   right	   right	   right	    down	
    down	    down	    left	    left	      up	      up	      up	
   right	      up	    left	    left	    down	    left	    left	
   right	    left	    left	    left	      up	      up	    left	
----------------------------------------------------------------------------------------------------
Frontier at step 1

    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	

{(4, 0): 'up',
 (3, 4): 'up',
 (4, 3): 'up',
 (3, 1): 'up',
 (5, 4): 'ouch!',
 (4, 6): 'right',
 (5, 1): 'up',
 (0, 2): 'up',
 (0, 5): 'right',
 (2, 2): 'up',
 (1, 0): 'up',
 (1, 6): 'right',
 (2, 5): 'right',
 (1, 3): 'right',
 (6, 2): 'up',
 (6, 5): 'left',
 (4, 2): 'up',
 (3, 0): 'right',
 (4, 5): 'right',
 (3, 3): 'right',
 (5, 0): 'up',
 (5, 6): 'down',
 (3, 6): 'right',
 (5, 3): 'right',
 (0, 1): 'right',
 (2, 4): 'up',
 (1, 2): 'up',
 (0, 4): 'right',
 (2, 1): 'up',
 (1, 5): 'right',
 (6, 1): 'up',
 (6, 4): 'up',
 (3, 2): 'up',
 (4, 1): 'up',
 (3, 5): 'right',
 (5, 2): 'up',
 (4, 4): 'up',
 (5, 5): 'catch!',
 (0, 0): 'up',
 (1, 1): 'up',
 (0, 3): 'right',
 (2, 0): 'right',
 (1, 4): 'right',
 (0, 6): 'right',
 (2, 3): 'right',
 (2, 6): 'right',
 (6, 0): 'up',
 (6, 6): 'down',
 (6, 3): 'up'}

In [ ]:
q = deterministic_q_value_iteration(tom_and_jerry)
deterministic_policy_from_q(tom_and_jerry, q, verbose=True)

Frontier at step 1

    down	    down	    down	    down	    down	    down	    down	
   right	   right	   right	   right	   right	  catch!	    left	
      up	      up	      up	      up	      up	   ouch!	      up	
      up	      up	      up	      up	      up	    left	      up	
      up	      up	      up	      up	      up	      up	      up	
      up	      up	      up	      up	      up	      up	      up	
      up	      up	      up	      up	      up	      up	      up	
----------------------------------------------------------------------------------------------------


{(4, 0): 'up',
 (3, 4): 'up',
 (4, 3): 'up',
 (3, 1): 'up',
 (5, 4): 'ouch!',
 (4, 6): 'down',
 (5, 1): 'up',
 (0, 2): 'up',
 (0, 5): 'right',
 (2, 2): 'up',
 (1, 0): 'up',
 (1, 6): 'down',
 (2, 5): 'right',
 (1, 3): 'up',
 (6, 2): 'up',
 (6, 5): 'left',
 (4, 2): 'up',
 (3, 0): 'up',
 (4, 5): 'right',
 (3, 3): 'up',
 (5, 0): 'up',
 (5, 6): 'down',
 (3, 6): 'down',
 (5, 3): 'left',
 (0, 1): 'up',
 (2, 4): 'up',
 (1, 2): 'up',
 (0, 4): 'up',
 (2, 1): 'up',
 (1, 5): 'right',
 (6, 1): 'up',
 (6, 4): 'up',
 (3, 2): 'up',
 (4, 1): 'up',
 (3, 5): 'right',
 (5, 2): 'up',
 (4, 4): 'up',
 (5, 5): 'catch!',
 (0, 0): 'up',
 (1, 1): 'up',
 (0, 3): 'up',
 (2, 0): 'up',
 (1, 4): 'up',
 (0, 6): 'down',
 (2, 3): 'up',
 (2, 6): 'down',
 (6, 0): 'up',
 (6, 6): 'down',
 (6, 3): 'up'}

**SELF-CHECK:** What would happen if $\gamma = 1$?

## Markovian Decision Processes (MDPs)

Great! Now we have yet another way to solve deterministic classical search problems! Why bother? As we mentioned earlier, this is just an introduction to decision processes. In the real world, actions are _far from deterministic_; time passes, weather changes, actuators fail, etc. Even in our simple Tom & Jerry world, as anyone who have watched the show would notice, Tom slips... A LOT. A very famous way to model this stochasticity is Markovian Decision Processes (MDPs).

In [ ]:
from IPython.display import Image
Image(url='https://media.giphy.com/media/47xzuq4tImaV0ski1T/giphy-downsized-large.gif')

In an MDP, we assume the new state resulting from applying an action to a state depends only on the state and the action. That is, it does not depend on previous states. While this may seem like a reasonable assumption, it is usually the case, but it does simplify the model considerably. (It is called the Markovian assumption and it is what the _M_ in _MDP_ stands for.)

Using this assumption, we only need to change our _result_ function to be _results_! A function that returns a _distribution_ over the possible next states. (This is called the _transition model_ by the way.)

In [ ]:
class MDP:
    '''
    Abstract base class for Markovian Decision Process (MDP) formulation.
    It declares the expected methods to be used to solve it.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''

    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the state space.'''
        self.states = set()

    def actions(self, state):
        '''Returns an iterable with the applicable actions to the given state.'''
        raise NotImplementedError

    def results(self, state, action):
        '''Returns a ditribution over all possible resulting states from applying the given action to the given state.'''
        raise NotImplementedError

    def reward(self, state):
        '''Returns the reward of being in a certain state'''
        raise NotImplementedError

Next, we need to modify our previous algorithms to replace anything depending on _the next state_ with a summation over the distribution over all possible next states... and that's it! Here are the famous algorithms to solve an MDP: **value iteration**, **policy iteration**, and **Q-value iteration**!

### Value Iteration

$$U_{i+1}(s) = r_s + \gamma \times \max_{a \in A_s} \sum_{s'} P(s'|s,a) U_{i}(s')$$

$$\pi^*(s) = \argmax_{a \in A_s} \sum_{s'} P(s'|s,a) U_{i}(s')$$

In [ ]:
def value_iteration(mdp, init_utility=None, error=1e-6, verbose=False):
    if verbose: visualizer = Visualizer(mdp)
    utility = init_utility if init_utility is not None else {state: 0 for state in mdp.states | {None}}
    converged = False
    while not converged:
        if verbose: visualizer.visualize([utility])
        old_utility, converged = utility.copy(), True
        for state in mdp.states:
            utility[state] = mdp.reward(state) + mdp.discount * max(sum(p * old_utility[next_state] for next_state, p in mdp.results(state, action).items()) for action in mdp.actions(state))
            if abs(utility[state] - old_utility[state]) > error: converged = False
    return utility

def policy_from_values(mdp, utility, verbose=True):
    if verbose: visualizer = Visualizer(mdp)
    policy = {}
    for state in mdp.states:
        policy[state] = max((action for action in mdp.actions(state)), key=lambda action: sum(p * utility[next_state] for next_state, p in mdp.results(state, action).items()))
    if verbose: visualizer.visualize([policy])
    return policy

### Policy Iteration

$$U_{i+1}(s) = r_s + \gamma \times \sum_{s'} P(s'|s,a) U_{i}(s')$$

$$\pi_{i+1}(s) = \argmax_{a \in A_s} \sum_{s'} P(s'|s,a) U_{i+1}(s')$$

In [ ]:
from random import choice

def policy_evaluation(mdp, policy, init_utility=None, error=1e-6, verbose=False):
    if verbose: visualizer = Visualizer(mdp)
    utility = init_utility if init_utility is not None else {state: 0 for state in mdp.states | {None}}
    converged = False
    while not converged:
        if verbose: visualizer.visualize([utility])
        old_utility, converged = utility.copy(), True
        for state in mdp.states:
            utility[state] = mdp.reward(state) + mdp.discount * sum(p * old_utility[next_state] for next_state, p in mdp.results(state, policy[state]).items())
            if abs(utility[state] - old_utility[state]) > error: converged = False
    return utility

def policy_iteration(mdp, verbose=False):
    if verbose: visualizer = Visualizer(mdp)
    policy = {state: choice(mdp.actions(state)) for state in mdp.states}
    converged, utility = False, None
    while not converged:
        if verbose: visualizer.visualize([policy])
        converged = True
        utility = policy_evaluation(mdp, policy, utility, verbose=verbose)
        for state in mdp.states:
            for action in mdp.actions(state):
                if sum(p * utility[next_state] for next_state, p in mdp.results(state, action).items()) > sum(p * utility[next_state] for next_state, p in mdp.results(state, policy[state]).items()):
                    policy[state] = action
                    converged = False
    return policy

### Q-Value Iteration

$$Q_{i+1}(s, a) = r_s + \gamma \times \sum_{s'} P(s'|s,a) U_{i}(s')$$
$$\equiv$$
$$Q_{i+1}(s, a) = r_s + \gamma \times \sum_{s'} P(s'|s,a) \max_{a' \in A_{s'}} Q_{i}(s', a')$$

$$\pi^*(s) = \argmax_{a \in A_s} Q(s,a)$$

In [ ]:
def q_value_iteration(mdp, init_q=None, error=1e-6):
    q = init_q if init_q is not None else {(state, action): 0 for state in mdp.states for action in mdp.actions(state)}
    converged = False
    while not converged:
        old_q, converged = q.copy(), True
        for state in mdp.states:
            for action in mdp.actions(state):
                next_states = mdp.results(state, action)
                next_states_utility = sum(max((old_q[(next_state, next_action)] for next_action in mdp.actions(next_state))) * p for next_state, p in next_states.items())
                q[(state, action)] = mdp.reward(state) + mdp.discount * next_states_utility
                if abs(q[(state, action)] - old_q[(state, action)]) > error: converged = False
    return q

def policy_from_q(mdp, q, verbose=True):
    if verbose: visualizer = Visualizer(mdp)
    policy = {}
    for state in mdp.states:
        policy[state] = max((action for action in mdp.actions(state)), key=lambda action: q[(state, action)])
    if verbose: visualizer.visualize([policy])
    return policy

### Example: Slippery Tom & Jerry

Let's model the slippery motion in the above GIF. When Tom wants to move in a certain direction, he either moves or slips and stays put.

In [ ]:
class SlipperyTomAndJerry(MDP):
    '''Slippery Tom & Jerry world'''

    def __init__(self, tom, jerry, spike, bounds, max_reward, discount, p):
        self.tom = tom
        self.jerry = jerry
        self.spike = spike
        self.bounds = bounds
        self.max_reward = max_reward
        self.discount = discount
        self.p = p
        self.states = {(i, j) for i in range(bounds[0]) for j in range(bounds[1])}

    def actions(self, state):
        if state is None: return []
        if state == self.jerry: return ['catch!']
        if state == self.spike: return ['ouch!']
        return ['up', 'down', 'left', 'right']

    def results(self, state, action):
        if   action ==    'up': return {(state[0], min(state[1] + 1, self.bounds[1] - 1)): 1.0-self.p, state: self.p}
        elif action ==  'down': return {(state[0], max(state[1] - 1, 0)): 1.0-self.p, state: self.p}
        elif action ==  'left': return {(max(state[0] - 1, 0), state[1]): 1.0-self.p, state: self.p}
        elif action == 'right': return {(min(state[0] + 1, self.bounds[0] - 1), state[1]): 1.0-self.p, state: self.p}
        else: return {}

    def reward(self, state):
        if state == self.jerry: return  self.max_reward
        if state == self.spike: return -self.max_reward
        return 0

_visualizers[SlipperyTomAndJerry] = _tom_and_jerry_visualizer

In [ ]:
slippery_tom_and_jerry = SlipperyTomAndJerry((0, 0), (5, 5), (5, 4), (7, 7), 1000, 0.1, 0.1)

In [ ]:
values = value_iteration(slippery_tom_and_jerry, verbose=True)
policy_from_values(slippery_tom_and_jerry, values, verbose=True)

Frontier at step 1

    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
----------------------------------------------------------------------------------------------------
Frontier at step 2

    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	 1000.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	-1000.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	

{(4, 0): 'up',
 (3, 4): 'up',
 (4, 3): 'up',
 (3, 1): 'up',
 (5, 4): 'ouch!',
 (4, 6): 'down',
 (5, 1): 'up',
 (0, 2): 'up',
 (0, 5): 'right',
 (2, 2): 'up',
 (1, 0): 'up',
 (1, 6): 'down',
 (2, 5): 'right',
 (1, 3): 'up',
 (6, 2): 'up',
 (6, 5): 'left',
 (4, 2): 'up',
 (3, 0): 'up',
 (4, 5): 'right',
 (3, 3): 'up',
 (5, 0): 'up',
 (5, 6): 'down',
 (3, 6): 'down',
 (5, 3): 'left',
 (0, 1): 'up',
 (2, 4): 'up',
 (1, 2): 'up',
 (0, 4): 'up',
 (2, 1): 'up',
 (1, 5): 'right',
 (6, 1): 'up',
 (6, 4): 'up',
 (3, 2): 'up',
 (4, 1): 'up',
 (3, 5): 'right',
 (5, 2): 'up',
 (4, 4): 'up',
 (5, 5): 'catch!',
 (0, 0): 'up',
 (1, 1): 'up',
 (0, 3): 'up',
 (2, 0): 'up',
 (1, 4): 'up',
 (0, 6): 'down',
 (2, 3): 'up',
 (2, 6): 'down',
 (6, 0): 'up',
 (6, 6): 'down',
 (6, 3): 'up'}

In [ ]:
policy_iteration(slippery_tom_and_jerry, verbose=True)

Frontier at step 1

   right	   right	    down	      up	   right	   right	    down	
    down	    down	    down	    left	    down	  catch!	    down	
      up	      up	    down	    left	    left	   ouch!	    down	
      up	    down	    down	    down	   right	    left	    left	
    left	   right	   right	   right	   right	    down	      up	
    left	    left	    left	   right	      up	      up	   right	
   right	    down	      up	      up	    left	    down	    left	
----------------------------------------------------------------------------------------------------
Frontier at step 1

    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	

{(4, 0): 'up',
 (3, 4): 'up',
 (4, 3): 'up',
 (3, 1): 'right',
 (5, 4): 'ouch!',
 (4, 6): 'right',
 (5, 1): 'up',
 (0, 2): 'right',
 (0, 5): 'right',
 (2, 2): 'right',
 (1, 0): 'up',
 (1, 6): 'right',
 (2, 5): 'right',
 (1, 3): 'up',
 (6, 2): 'up',
 (6, 5): 'left',
 (4, 2): 'up',
 (3, 0): 'up',
 (4, 5): 'right',
 (3, 3): 'up',
 (5, 0): 'up',
 (5, 6): 'down',
 (3, 6): 'right',
 (5, 3): 'left',
 (0, 1): 'up',
 (2, 4): 'up',
 (1, 2): 'right',
 (0, 4): 'right',
 (2, 1): 'up',
 (1, 5): 'right',
 (6, 1): 'up',
 (6, 4): 'up',
 (3, 2): 'right',
 (4, 1): 'up',
 (3, 5): 'right',
 (5, 2): 'up',
 (4, 4): 'up',
 (5, 5): 'catch!',
 (0, 0): 'right',
 (1, 1): 'up',
 (0, 3): 'up',
 (2, 0): 'right',
 (1, 4): 'up',
 (0, 6): 'right',
 (2, 3): 'up',
 (2, 6): 'right',
 (6, 0): 'up',
 (6, 6): 'down',
 (6, 3): 'up'}

In [ ]:
q = q_value_iteration(slippery_tom_and_jerry)
policy_from_q(slippery_tom_and_jerry, q, verbose=True)

Frontier at step 1

    down	    down	    down	    down	    down	    down	    down	
   right	   right	   right	   right	   right	  catch!	    left	
      up	      up	      up	      up	      up	   ouch!	      up	
      up	      up	      up	      up	      up	    left	      up	
      up	      up	      up	      up	      up	      up	      up	
      up	      up	      up	      up	      up	      up	      up	
      up	      up	      up	      up	      up	      up	      up	
----------------------------------------------------------------------------------------------------


{(4, 0): 'up',
 (3, 4): 'up',
 (4, 3): 'up',
 (3, 1): 'up',
 (5, 4): 'ouch!',
 (4, 6): 'down',
 (5, 1): 'up',
 (0, 2): 'up',
 (0, 5): 'right',
 (2, 2): 'up',
 (1, 0): 'up',
 (1, 6): 'down',
 (2, 5): 'right',
 (1, 3): 'up',
 (6, 2): 'up',
 (6, 5): 'left',
 (4, 2): 'up',
 (3, 0): 'up',
 (4, 5): 'right',
 (3, 3): 'up',
 (5, 0): 'up',
 (5, 6): 'down',
 (3, 6): 'down',
 (5, 3): 'left',
 (0, 1): 'up',
 (2, 4): 'up',
 (1, 2): 'up',
 (0, 4): 'up',
 (2, 1): 'up',
 (1, 5): 'right',
 (6, 1): 'up',
 (6, 4): 'up',
 (3, 2): 'up',
 (4, 1): 'up',
 (3, 5): 'right',
 (5, 2): 'up',
 (4, 4): 'up',
 (5, 5): 'catch!',
 (0, 0): 'up',
 (1, 1): 'up',
 (0, 3): 'up',
 (2, 0): 'up',
 (1, 4): 'up',
 (0, 6): 'down',
 (2, 3): 'up',
 (2, 6): 'down',
 (6, 0): 'up',
 (6, 6): 'down',
 (6, 3): 'up'}

**SELF-CHECK:** What would happen if $p = 0$?

In [ ]:
class MoreSlipperyTomAndJerry(MDP):
    '''More Slippery Tom & Jerry world'''

    def __init__(self, tom, jerry, spike, bounds, max_reward, discount, p):
        self.tom = tom
        self.jerry = jerry
        self.spike = spike
        self.bounds = bounds
        self.max_reward = max_reward
        self.discount = discount
        self.p = p
        self.states = {(i, j) for i in range(bounds[0]) for j in range(bounds[1])}

    def actions(self, state):
        if state is None: return []
        if state == self.jerry: return ['catch!']
        if state == self.spike: return ['ouch!']
        return ['up', 'down', 'left', 'right']

    def results(self, state, action):
        up_motion = (state[0], min(state[1] + 1, self.bounds[1] - 1))
        down_motion = (state[0], max(state[1] - 1, 0))
        left_motion = (max(state[0] - 1, 0), state[1])
        right_motion = (min(state[0] + 1, self.bounds[0] - 1), state[1])
        if   action ==    'up': return {up_motion: 1.0-2.0*self.p, left_motion: self.p, right_motion: self.p}
        elif action ==  'down': return {down_motion: 1.0-2.0*self.p, left_motion: self.p, right_motion: self.p}
        elif action ==  'left': return {left_motion: 1.0-2.0*self.p, up_motion: self.p, down_motion: self.p}
        elif action == 'right': return {right_motion: 1.0-2.0*self.p, up_motion: self.p, down_motion: self.p}
        else: return {}

    def reward(self, state):
        if state == self.jerry: return  self.max_reward

        if state == self.spike: return -self.max_reward

        return 0

_visualizers[MoreSlipperyTomAndJerry] = _tom_and_jerry_visualizer

In [ ]:
more_slippery_tom_and_jerry = MoreSlipperyTomAndJerry((0, 0), (5, 5),(5, 4), (7, 7), 1000, 0.1, 0.1)
values = value_iteration(more_slippery_tom_and_jerry, verbose=True)
policy = policy_from_values(more_slippery_tom_and_jerry, values, verbose=True)
Visualizer(more_slippery_tom_and_jerry).visualize([values]),
Visualizer(more_slippery_tom_and_jerry).visualize([policy])

Frontier at step 1

    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
----------------------------------------------------------------------------------------------------
Frontier at step 2

    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	 1000.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	-1000.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	    0.00	
    0.00	    0.00	    0.00	

## Requirement

You only have to do **TWO** of the following parts.


### Part A

As you can see, the algorithms work irrespective of how complex the transition model is. Try adding obstacles in Tom's way and see how the algorithms will behave! This would be similar to the _2D Maze_ formulation back in classical search, but not quite the same.

### Part B

We used the _discount_ to motivate Tom to catch Jerry as fast as possible. Another way to do so is to assign a negative reward to moving around needlessly. Try adding that to the formulation and see how the value of the negative rewards affects the policy.

What happens if moving around is too painful (say, Tom's foot is injured)? Can you replicate that effect with the _discount_? Why or why not?

### Part C

Can you make Jerry move? What would the new state space look like? Try writing code to capture such a case and see how the algorithms would solve it!

You can have Jerry moving in any way you like (even random!) as long as you can model it with the _results_ function.

**Estimated time for this exercise is 2 hours!**

# **Part A**

In [ ]:
class SlipperyTomAndJerry_obstacles(MDP):
    '''Slippery Tom & Jerry world'''
    def __init__(self, tom, jerry, spike, bounds, max_reward, discount, p, Obstacles):
        self.tom = tom
        self.jerry = jerry
        self.spike = spike
        self.bounds = bounds
        self.max_reward = max_reward
        self.discount = discount
        self.p = p
        self.states = {(i, j) for i in range(bounds[0]) for j in range(bounds[1])}
        self.Obstacles = Obstacles;

    def actions(self, state):
        if state is None: return []
        if state == self.jerry: return ['catch!']
        if state == self.spike: return ['ouch!']
        if [state[0]-1,state[1]] in self.Obstacles: return ['down', 'left', 'right']
        if [state[0]+1,state[1]] in self.Obstacles: return ['up', 'left', 'right']
        if [state[0],state[1]+1] in self.Obstacles: return ['down', 'left', 'up']
        if [state[0],state[1]-1] in self.Obstacles: return ['down', 'up', 'right']
        return ['up', 'down', 'left', 'right']

    def results(self, state, action):
        if   action ==    'up': return {(state[0], min(state[1] + 1, self.bounds[1] - 1)): 1.0-self.p, state: self.p}
        elif action ==  'down': return {(state[0], max(state[1] - 1, 0)): 1.0-self.p, state: self.p}
        elif action ==  'left': return {(max(state[0] - 1, 0), state[1]): 1.0-self.p, state: self.p}
        elif action == 'right': return {(min(state[0] + 1, self.bounds[0] - 1), state[1]): 1.0-self.p, state: self.p}
        else: return {}

    def reward(self, state):
        if state == self.jerry: return  self.max_reward
        if state == self.spike: return -self.max_reward
        return 0

_visualizers[SlipperyTomAndJerry] = _tom_and_jerry_visualizer

In [ ]:
obstacles = [ [3,3] , [5,7], [8,8]]
slippery = SlipperyTomAndJerry_obstacles((0, 0), (5, 5), (5, 4), (7, 7), 1000, 0.1, 0.1,obstacles)

In [ ]:
values = value_iteration(slippery, verbose=True)
policy_from_values(slippery, values, verbose=True)

Frontier at step 1

{(4, 0): 0, (3, 4): 0, (4, 3): 0, (3, 1): 0, (5, 4): 0, (4, 6): 0, (5, 1): 0, (0, 2): 0, (0, 5): 0, (2, 2): 0, (1, 0): 0, (1, 6): 0, (2, 5): 0, (1, 3): 0, (6, 2): 0, (6, 5): 0, (4, 2): 0, (3, 0): 0, (4, 5): 0, (3, 3): 0, (5, 0): 0, (5, 6): 0, (3, 6): 0, (5, 3): 0, None: 0, (0, 1): 0, (2, 4): 0, (1, 2): 0, (0, 4): 0, (2, 1): 0, (1, 5): 0, (6, 1): 0, (6, 4): 0, (3, 2): 0, (4, 1): 0, (3, 5): 0, (5, 2): 0, (4, 4): 0, (5, 5): 0, (0, 0): 0, (1, 1): 0, (0, 3): 0, (2, 0): 0, (1, 4): 0, (0, 6): 0, (2, 3): 0, (2, 6): 0, (6, 0): 0, (6, 6): 0, (6, 3): 0}
----------------------------------------------------------------------------------------------------
Frontier at step 2

{(4, 0): 0.0, (3, 4): 0.0, (4, 3): 0.0, (3, 1): 0.0, (5, 4): -1000.0, (4, 6): 0.0, (5, 1): 0.0, (0, 2): 0.0, (0, 5): 0.0, (2, 2): 0.0, (1, 0): 0.0, (1, 6): 0.0, (2, 5): 0.0, (1, 3): 0.0, (6, 2): 0.0, (6, 5): 0.0, (4, 2): 0.0, (3, 0): 0.0, (4, 5): 0.0, (3, 3): 0.0, (5, 0): 0.0, (5, 6): 0.0, (3, 6): 0.0, (5, 3)

{(4, 0): 'up',
 (3, 4): 'up',
 (4, 3): 'left',
 (3, 1): 'up',
 (5, 4): 'ouch!',
 (4, 6): 'down',
 (5, 1): 'up',
 (0, 2): 'up',
 (0, 5): 'right',
 (2, 2): 'up',
 (1, 0): 'up',
 (1, 6): 'down',
 (2, 5): 'right',
 (1, 3): 'up',
 (6, 2): 'up',
 (6, 5): 'left',
 (4, 2): 'up',
 (3, 0): 'up',
 (4, 5): 'right',
 (3, 3): 'up',
 (5, 0): 'up',
 (5, 6): 'down',
 (3, 6): 'down',
 (5, 3): 'right',
 (0, 1): 'up',
 (2, 4): 'up',
 (1, 2): 'up',
 (0, 4): 'up',
 (2, 1): 'up',
 (1, 5): 'right',
 (6, 1): 'up',
 (6, 4): 'up',
 (3, 2): 'up',
 (4, 1): 'up',
 (3, 5): 'right',
 (5, 2): 'up',
 (4, 4): 'up',
 (5, 5): 'catch!',
 (0, 0): 'up',
 (1, 1): 'up',
 (0, 3): 'up',
 (2, 0): 'up',
 (1, 4): 'up',
 (0, 6): 'down',
 (2, 3): 'up',
 (2, 6): 'down',
 (6, 0): 'up',
 (6, 6): 'down',
 (6, 3): 'up'}

In [ ]:
policy_iteration(slippery, verbose=True)

Frontier at step 1

{(4, 0): 'left', (3, 4): 'up', (4, 3): 'left', (3, 1): 'left', (5, 4): 'ouch!', (4, 6): 'up', (5, 1): 'right', (0, 2): 'right', (0, 5): 'down', (2, 2): 'right', (1, 0): 'right', (1, 6): 'up', (2, 5): 'down', (1, 3): 'up', (6, 2): 'down', (6, 5): 'right', (4, 2): 'up', (3, 0): 'left', (4, 5): 'up', (3, 3): 'left', (5, 0): 'down', (5, 6): 'left', (3, 6): 'down', (5, 3): 'down', (0, 1): 'up', (2, 4): 'up', (1, 2): 'down', (0, 4): 'up', (2, 1): 'down', (1, 5): 'up', (6, 1): 'down', (6, 4): 'right', (3, 2): 'up', (4, 1): 'down', (3, 5): 'right', (5, 2): 'right', (4, 4): 'right', (5, 5): 'catch!', (0, 0): 'right', (1, 1): 'left', (0, 3): 'down', (2, 0): 'left', (1, 4): 'left', (0, 6): 'down', (2, 3): 'up', (2, 6): 'left', (6, 0): 'right', (6, 6): 'down', (6, 3): 'down'}
----------------------------------------------------------------------------------------------------
Frontier at step 1

{(4, 0): 0, (3, 4): 0, (4, 3): 0, (3, 1): 0, (5, 4): 0, (4, 6): 0, (5, 1): 0, (0, 2)

{(4, 0): 'up',
 (3, 4): 'up',
 (4, 3): 'left',
 (3, 1): 'up',
 (5, 4): 'ouch!',
 (4, 6): 'down',
 (5, 1): 'up',
 (0, 2): 'right',
 (0, 5): 'right',
 (2, 2): 'right',
 (1, 0): 'up',
 (1, 6): 'right',
 (2, 5): 'right',
 (1, 3): 'right',
 (6, 2): 'up',
 (6, 5): 'left',
 (4, 2): 'up',
 (3, 0): 'up',
 (4, 5): 'right',
 (3, 3): 'up',
 (5, 0): 'up',
 (5, 6): 'down',
 (3, 6): 'down',
 (5, 3): 'right',
 (0, 1): 'up',
 (2, 4): 'right',
 (1, 2): 'right',
 (0, 4): 'right',
 (2, 1): 'up',
 (1, 5): 'right',
 (6, 1): 'up',
 (6, 4): 'up',
 (3, 2): 'up',
 (4, 1): 'up',
 (3, 5): 'right',
 (5, 2): 'up',
 (4, 4): 'up',
 (5, 5): 'catch!',
 (0, 0): 'up',
 (1, 1): 'up',
 (0, 3): 'right',
 (2, 0): 'up',
 (1, 4): 'right',
 (0, 6): 'right',
 (2, 3): 'up',
 (2, 6): 'right',
 (6, 0): 'up',
 (6, 6): 'down',
 (6, 3): 'up'}

In [ ]:
q = q_value_iteration(slippery)
policy_from_q(slippery, q, verbose=True)

Frontier at step 1

{(4, 0): 'up', (3, 4): 'up', (4, 3): 'left', (3, 1): 'up', (5, 4): 'ouch!', (4, 6): 'down', (5, 1): 'up', (0, 2): 'up', (0, 5): 'right', (2, 2): 'up', (1, 0): 'up', (1, 6): 'down', (2, 5): 'right', (1, 3): 'up', (6, 2): 'up', (6, 5): 'left', (4, 2): 'up', (3, 0): 'up', (4, 5): 'right', (3, 3): 'up', (5, 0): 'up', (5, 6): 'down', (3, 6): 'down', (5, 3): 'right', (0, 1): 'up', (2, 4): 'up', (1, 2): 'up', (0, 4): 'up', (2, 1): 'up', (1, 5): 'right', (6, 1): 'up', (6, 4): 'up', (3, 2): 'up', (4, 1): 'up', (3, 5): 'right', (5, 2): 'up', (4, 4): 'up', (5, 5): 'catch!', (0, 0): 'up', (1, 1): 'up', (0, 3): 'up', (2, 0): 'up', (1, 4): 'up', (0, 6): 'down', (2, 3): 'up', (2, 6): 'down', (6, 0): 'up', (6, 6): 'down', (6, 3): 'up'}
----------------------------------------------------------------------------------------------------


{(4, 0): 'up',
 (3, 4): 'up',
 (4, 3): 'left',
 (3, 1): 'up',
 (5, 4): 'ouch!',
 (4, 6): 'down',
 (5, 1): 'up',
 (0, 2): 'up',
 (0, 5): 'right',
 (2, 2): 'up',
 (1, 0): 'up',
 (1, 6): 'down',
 (2, 5): 'right',
 (1, 3): 'up',
 (6, 2): 'up',
 (6, 5): 'left',
 (4, 2): 'up',
 (3, 0): 'up',
 (4, 5): 'right',
 (3, 3): 'up',
 (5, 0): 'up',
 (5, 6): 'down',
 (3, 6): 'down',
 (5, 3): 'right',
 (0, 1): 'up',
 (2, 4): 'up',
 (1, 2): 'up',
 (0, 4): 'up',
 (2, 1): 'up',
 (1, 5): 'right',
 (6, 1): 'up',
 (6, 4): 'up',
 (3, 2): 'up',
 (4, 1): 'up',
 (3, 5): 'right',
 (5, 2): 'up',
 (4, 4): 'up',
 (5, 5): 'catch!',
 (0, 0): 'up',
 (1, 1): 'up',
 (0, 3): 'up',
 (2, 0): 'up',
 (1, 4): 'up',
 (0, 6): 'down',
 (2, 3): 'up',
 (2, 6): 'down',
 (6, 0): 'up',
 (6, 6): 'down',
 (6, 3): 'up'}

# **PART B:**

In [ ]:
class SlipperyTomAndJerry_NegativeReward(MDP):
    '''Slippery Tom & Jerry world'''
    def __init__(self, tom, jerry, spike, bounds, max_reward, discount, p):
        self.tom = tom
        self.jerry = jerry
        self.spike = spike
        self.bounds = bounds
        self.max_reward = max_reward
        self.discount = discount
        self.p = p
        self.states = {(i, j) for i in range(bounds[0]) for j in range(bounds[1])}
    def actions(self, state):
        if state is None: return []
        if state == self.jerry: return ['catch!']
        if state == self.spike: return ['ouch!']
        return ['up', 'down', 'left', 'right']
    def results(self, state, action):
        if   action ==    'up': return {(state[0], min(state[1] + 1, self.bounds[1] - 1)): 1.0-self.p, state: self.p}
        elif action ==  'down': return {(state[0], max(state[1] - 1, 0)): 1.0-self.p, state: self.p}
        elif action ==  'left': return {(max(state[0] - 1, 0), state[1]): 1.0-self.p, state: self.p}
        elif action == 'right': return {(min(state[0] + 1, self.bounds[0] - 1), state[1]): 1.0-self.p, state: self.p}
        else: return {}

    def reward(self, state):
        if state == self.jerry: return  self.max_reward
        if state == self.spike: return -self.max_reward
        return -1

_visualizers[SlipperyTomAndJerry] = _tom_and_jerry_visualizer

In [ ]:
slippery_tom_and_jerry_NegativeReward = SlipperyTomAndJerry_NegativeReward((0, 0), (5, 5), (5, 4), (7, 7), 1000, 0.1, 0.1)
values = value_iteration(slippery_tom_and_jerry_NegativeReward, verbose=True)
policy = policy_from_values(slippery_tom_and_jerry_NegativeReward, values, verbose=True)
Visualizer(slippery_tom_and_jerry_NegativeReward).visualize([values])
Visualizer(slippery_tom_and_jerry_NegativeReward).visualize([policy])

Frontier at step 1

{(4, 0): 0, (3, 4): 0, (4, 3): 0, (3, 1): 0, (5, 4): 0, (4, 6): 0, (5, 1): 0, (0, 2): 0, (0, 5): 0, (2, 2): 0, (1, 0): 0, (1, 6): 0, (2, 5): 0, (1, 3): 0, (6, 2): 0, (6, 5): 0, (4, 2): 0, (3, 0): 0, (4, 5): 0, (3, 3): 0, (5, 0): 0, (5, 6): 0, (3, 6): 0, (5, 3): 0, None: 0, (0, 1): 0, (2, 4): 0, (1, 2): 0, (0, 4): 0, (2, 1): 0, (1, 5): 0, (6, 1): 0, (6, 4): 0, (3, 2): 0, (4, 1): 0, (3, 5): 0, (5, 2): 0, (4, 4): 0, (5, 5): 0, (0, 0): 0, (1, 1): 0, (0, 3): 0, (2, 0): 0, (1, 4): 0, (0, 6): 0, (2, 3): 0, (2, 6): 0, (6, 0): 0, (6, 6): 0, (6, 3): 0}
----------------------------------------------------------------------------------------------------
Frontier at step 2

{(4, 0): -1.0, (3, 4): -1.0, (4, 3): -1.0, (3, 1): -1.0, (5, 4): -1000.0, (4, 6): -1.0, (5, 1): -1.0, (0, 2): -1.0, (0, 5): -1.0, (2, 2): -1.0, (1, 0): -1.0, (1, 6): -1.0, (2, 5): -1.0, (1, 3): -1.0, (6, 2): -1.0, (6, 5): -1.0, (4, 2): -1.0, (3, 0): -1.0, (4, 5): -1.0, (3, 3): -1.0, (5, 0): -1.0, (5, 6): -1.0

In [ ]:
from random import choice, randrange
class JerryMoving(MDP):
    def __init__(self, tom, jerry, spike, bounds, max_reward, discount, p):
        self.tom = tom
        self.jerry = jerry
        self.spike = spike
        self.bounds = bounds
        self.max_reward = max_reward
        self.discount = discount
        self.p = p
        self.states = {(i, j) for i in range(bounds[0]) for j in range(bounds[1])}

    def actions(self,state):
        if state is None: return []
        if state == self.jerry: return ['catch!']
        if state ==  self.spike: return ['ouch!']
        return ['up', 'down', 'left', 'right']

    def results(self, state, action):
        up = lambda position: (position[0], min(position[1] + 1, self.bounds[1] - 1))
        down = lambda position: (position[0], max(position[1] - 1, 0))
        left = lambda position: (max(position[0] - 1, 0), position[1])
        right = lambda position: (min(position[0] + 1, self.bounds[0] - 1), position[1])
        Jerry = choice((up, down, left, right));
        self.jerry = Jerry(self.jerry);
        if   action ==    'up': return {(state[0], min(state[1] + 1, self.bounds[1] - 1)): 1.0-self.p, state: self.p}
        elif action ==  'down': return {(state[0], max(state[1] - 1, 0)): 1.0-self.p, state: self.p}
        elif action ==  'left': return {(max(state[0] - 1, 0), state[1]): 1.0-self.p, state: self.p}
        elif action == 'right': return {(min(state[0] + 1, self.bounds[0] - 1), state[1]): 1.0-self.p, state: self.p}
        else: return {}

    def reward(self, state):
        if state == self.jerry: return  self.max_reward
        if state == self.spike: return -self.max_reward
        return 0;

    @classmethod
    def new_random_instance(cls, bounds, max_reward, discount):
        random_pair = lambda: (randrange(bounds[0]), randrange(bounds[1]))
        return cls(random_pair(), random_pair(), random_pair(), bounds, max_reward, discount)


_visualizers[JerryMoving] = _tom_and_jerry_visualizer

In [ ]:
JERRY = JerryMoving((0, 0), (5, 5), (5, 4), (7, 7), 1000, 0.1, 0.1)
values = value_iteration(JERRY, verbose=True)
policy = policy_from_values(JERRY, values, verbose=True)
Visualizer(JERRY).visualize([values])
Visualizer(JERRY).visualize([policy])

Streaming output truncated to the last 5000 lines.
Frontier at step 19590

    0.01	    0.00	    0.01	    0.00	    0.01	    0.00	    0.01	
    8.10	    0.01	    0.00	    0.01	   90.00	    0.01	    0.00	
    1.81	    8.10	    0.01	   90.00	   10.01	-1000.00	    0.01	
    8.20	    1.81	    8.10	    0.01	   90.00	    0.01	    0.00	
   90.90	    8.10	    0.01	   90.00	    0.01	    0.00	    0.01	
   18.10	   90.00	   90.00	   10.01	   90.00	    0.01	    0.00	
   90.00	    0.00	    0.01	   90.00	    0.01	    0.00	    0.00	
----------------------------------------------------------------------------------------------------
Frontier at step 19591

    0.73	    0.00	    0.00	 1000.00	    8.10	    0.00	    0.00	
    0.24	    0.73	    0.00	    8.10	    1.80	    8.10	    0.00	
    0.76	    0.24	    8.10	    1.80	    8.20	-1000.00	    0.00	
    8.26	    0.76	    0.24	    8.10	    1.80	    8.10	    0.00	
    2.54	    8.26	    8.10	    1.80	    8.10	    0.00	    0.00	
    8.36	    9.00	    9.00	    8